# Make a movie of meteorology

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from palettable.colorbrewer.diverging import RdBu_11
from palettable.colorbrewer.sequential import *
import glob
import moviepy.editor as mpy

In [2]:
# Alpine-3D grids
ds = xr.open_dataset("../output/grids/a3d_grids.nc")

# SNOWPACK topography 
dem = np.flipud(np.loadtxt("../input/surface-grids/dem.asc", skiprows=6))
dem = xr.DataArray(dem, coords=[ds['northing'], ds['easting']], dims=['northing', 'easting'])

# Trim grids
n_trim = 15
ds = ds.isel(easting=slice(n_trim, -n_trim))
ds = ds.isel(northing=slice(n_trim, -n_trim))
dem = dem.isel(easting=slice(n_trim, -n_trim))
dem = dem.isel(northing=slice(n_trim, -n_trim))

# Get eastins and northings
x_snowpack = ds['easting']
y_snowpack = ds['northing']

In [3]:
# Wind Speed
A3D_var = ds['ws']
colormap = Purples_9.mpl_colormap
title = "Wind Speed [m/s]"

# # SWE
# A3D_var = ds['swe'] - ds['swe'].isel(time=0)
# A3D_var = A3D_var.resample(time='6H').mean(dim='time') * 1000
# colormap = "viridis"
# title = "Surface Mass Balance [mm w.e.]"

# Time slice
n_0 = 7200
n_f = 7300
A3D_var = A3D_var.isel(time=slice(n_0, n_f))

# Get variable max/min
maxima = A3D_var.max()
minima = A3D_var.min()
print("Min = " + str(minima.values))
print("Max = " + str(maxima.values))
# maxima = 15
# minima = 0
# print("Min = " + str(minima))
# print("Max = " + str(maxima))

Min = 0.13664201
Max = 17.35582


In [4]:
# Clear old images, gifs,and movies
!mkdir -p movie_frame
!rm -f movie.gif
!rm -f movie.mp4
!rm -f movie_frame/*

In [5]:
# Make movie frames

for time_step in range(0, len(A3D_var['time']), 1):
# for time_step in range(0, 1, 1):

    # Plot map of mean wind
    print("Working on time step: " + str(time_step) + " of " + str(len(A3D_var['time'])))
    f = plt.figure(figsize=(30, 15))

    # DEM
    contour_levels = np.linspace(dem.min(), dem.max(), 25)
    contour = plt.contour(x_snowpack.values, y_snowpack.values, dem, contour_levels, linestyles='solid', colors='black')
    plt.clabel(contour, fontsize=20, fmt = '%.0f', inline=True)
    
    # Meteo variable
    plt.pcolor(x_snowpack.values, y_snowpack.values, A3D_var.isel(time=time_step), \
               cmap=colormap, vmin=minima, vmax=maxima)
    plt.title(str(A3D_var['time'][time_step].values)[0:19] + " " + title, fontsize=32)
    plt.xlabel("Easting", fontsize=32)
    plt.ylabel("Northing", fontsize=32)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=32)
    
    #Save Figure with image number zero padding 
    if time_step < 10: # 1 digit
        plt.savefig("movie_frame/frame_000" + str(time_step) + ".png", dpi=100)
    elif time_step < 100 and time_step > 9: # 2 digits
        plt.savefig("movie_frame/frame_00" + str(time_step) + ".png", dpi=100)
    elif time_step < 1000 and time_step > 99: # 3 digits
        plt.savefig("movie_frame/frame_0" + str(time_step) + ".png", dpi=100)
    else: # 4 digits
        plt.savefig("movie_frame/frame_" + str(time_step) + ".png", dpi=100)
    f.clear()
    plt.close(f)
    plt.close()

Working on time step: 0 of 100
Working on time step: 1 of 100
Working on time step: 2 of 100
Working on time step: 3 of 100
Working on time step: 4 of 100
Working on time step: 5 of 100
Working on time step: 6 of 100
Working on time step: 7 of 100
Working on time step: 8 of 100
Working on time step: 9 of 100
Working on time step: 10 of 100
Working on time step: 11 of 100
Working on time step: 12 of 100
Working on time step: 13 of 100
Working on time step: 14 of 100
Working on time step: 15 of 100
Working on time step: 16 of 100
Working on time step: 17 of 100
Working on time step: 18 of 100
Working on time step: 19 of 100
Working on time step: 20 of 100
Working on time step: 21 of 100
Working on time step: 22 of 100
Working on time step: 23 of 100
Working on time step: 24 of 100
Working on time step: 25 of 100
Working on time step: 26 of 100
Working on time step: 27 of 100
Working on time step: 28 of 100
Working on time step: 29 of 100
Working on time step: 30 of 100
Working on time st

In [7]:
# Make a .mp4 movie and gif
file_list = sorted(glob.glob('movie_frame/*.png'))
clip = mpy.ImageSequenceClip(file_list, fps=30)
clip.write_videofile('movie.mp4')
# clip.write_gif('movie.gif')

t:   2%|▏         | 2/100 [00:00<00:05, 18.85it/s, now=None]

Moviepy - Building video movie.mp4.
Moviepy - Writing video movie.mp4



Moviepy - Done !
Moviepy - video ready movie.mp4


In [ ]:
# Transect movie
northing_ind = 29
ymax = A3D_var[:,northing_ind,:].max()

# for time_step in range(0, len(A3D_var['time']), 2):
for time_step in range(0, 1, 1):

    # Plot map of mean wind
    print("Working on time step: " + str(time_step) + " of " + str(len(A3D_var['time'])))
    f = plt.figure(figsize=(30, 10))

    # Meteo variable
    A3D_var[time_step, northing_ind, :].plot()
    plt.grid()
    plt.ylim([0, ymax])
    
    #Save Figure with image number zero padding 
    if time_step < 10: # 1 digit
        plt.savefig("movie_frame/frame_000" + str(time_step) + ".png", dpi=100)
    elif time_step < 100 and time_step > 9: # 2 digits
        plt.savefig("movie_frame/frame_00" + str(time_step) + ".png", dpi=100)
    elif time_step < 1000 and time_step > 99: # 3 digits
        plt.savefig("movie_frame/frame_0" + str(time_step) + ".png", dpi=100)
    else: # 4 digits
        plt.savefig("movie_frame/frame_" + str(time_step) + ".png", dpi=100)
    f.clear()
    plt.close(f)
    plt.close()

In [ ]:
# Make a .mp4 movie and gif
file_list = sorted(glob.glob('movie_frame/*.png'))
clip = mpy.ImageSequenceClip(file_list, fps=15)
clip.write_videofile('movie.mp4')
# clip.write_gif('movie.gif')